In [1]:
import pyperclip
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [2]:
def get_from_google_scholar(scholar_url, paper_link=True, citation=True, title=True):
    option = Options()
    option.add_argument('--headless')
    driver = webdriver.Chrome(options=option)

    title_content = ""
    plink = ""
    bibtex_content = ""

    try:
        # Open the Google Scholar page
        driver.get(scholar_url)

        if title:
            title_h3 = driver.find_element(By.CSS_SELECTOR, "h3.gs_rt")
            title_a = title_h3.find_element(By.TAG_NAME, 'a')
            title_content = title_a.text
        if paper_link:
            try:
                pdf_div = driver.find_element(By.CSS_SELECTOR, "div.gs_or_ggsm")
                pdf_a = pdf_div.find_element(By.TAG_NAME, "a")
                plink = pdf_a.get_attribute('href')
            except:
                title_h3 = driver.find_element(By.CSS_SELECTOR, "h3.gs_rt")
                title_a = title_h3.find_element(By.TAG_NAME, 'a') 
                plink = title_a.get_attribute('href')
        if citation:
            # Locate the "Cite" button using its text content
            cite_button = driver.find_element(By.CSS_SELECTOR, "a.gs_or_cit.gs_or_btn.gs_nph")

            # Click the "Cite" button to reveal the citation options
            cite_button.click()
            wait = WebDriverWait(driver, 15)
            bibtex_a = wait.until(EC.visibility_of_all_elements_located((By.LINK_TEXT, 'BibTeX')))[0]

            # Locate the "BibTeX" option using its text content
            # bibtex_option = driver.find_element_by_xpath("//a[contains(text(), 'BibTeX')]")
            bibtex_link = bibtex_a.get_attribute('href')
            driver.get(bibtex_link)
            bibtex_content = driver.find_element(By.CSS_SELECTOR, 'pre').text
    finally:
        # Close the WebDriver
        driver.quit()
        return title_content, plink, bibtex_content

#### generate the file title

In [3]:
def gen_file_title(pr=True):
    title = input()

    title_template = f"""# {title}
*Here's some resources about {title}*"""
    if pr:
        print(title_template)
        pyperclip.copy(title_template)
        return

    return title_template

In [11]:
gen_file_title()

# Other DGMs
*Here's some resources about Other DGMs*


#### generate the resource title

In [4]:
def gen_resource_title(pr=True, resource_title=""):
    if resource_title == "":
        resource_title = input("title: ")
    read_tag = input("Have you read it? (yes/no, default no)")
    if read_tag.lower() == "no" or read_tag.lower() == "":
        read_tag = "UNREAD"
    elif read_tag.lower() == "yes":
        read_tag = "READ"
    else:
        read_tag = "UNKNOWN"
    resource_template = f"#### {resource_title} [`{read_tag}`]"

    if pr:
        print(resource_template)
        pyperclip.copy(resource_template)
        return

    return resource_template

In [96]:
gen_resource_title()

#### Video Diffusion Models [`READ`]


#### generate the book link with extraction code(baidu cloud disk)

In [5]:
# s = "链接: https://pan.baidu.com/s/1DeAIAIR_oUW9zLiX6eYc4A 提取码: e83d \n--来自百度网盘超级会员v2的分享"

def gen_baidu_cloud_link(pr=True):
    s = input()

    link = s.split(': ')[1].split()[0]
    ext_code = s.split(': ')[2].split()[0]
    template = f"book link: [here]({link}), with extraction code: `{ext_code}`"
    
    if pr:
        print(template)
        pyperclip.copy(template)
        return

    return template

In [38]:
gen_baidu_cloud_link()

book link: [here](https://pan.baidu.com/s/1ZhNKO0aEVWdHe-GEiJyn2g), with extraction code: `hahg`


#### generate the paper link

In [6]:
def gen_paper_link(pr=True, paper_link="", citation=""):
    if paper_link == "":
        paper_link = input("link: ")
    if citation == "":
        citation = input("citation: ")

    if paper_link == "" or citation == "":
        google_scholar_link = input("google scholar link: ")
        if google_scholar_link != "":
            _, p, c = get_from_google_scholar(google_scholar_link,
                                               paper_link=paper_link == "",
                                               citation=citation == "",
                                               title=False
                                               )
            if paper_link == "": paper_link = p
            if citation == "": citation = c

    paper_template = f"""
paper link: [here]({paper_link})

citation: 
```bibtex
{citation}
```
    """
    if pr:
        print(paper_template)
        pyperclip.copy(paper_template)
        return

    return paper_template

In [134]:
gen_paper_link()


paper link: [here](https://www.semanticscholar.org/paper/Understanding-the-exploding-gradient-problem-Pascanu-Mikolov/c5145b1d15fea9340840cc8bb6f0e46e8934827f)

citation: 
```bibtex

```
    


#### generate the notebook link

In [7]:
def gen_notebooke_link(pr=True):
    notebook_file = input("filename: ")
    notebook_theme = input("theme: ")
    notebook_template = f"here's a notebook about `{notebook_theme}`, including its formula and visualization: [here](../../notebooks/{notebook_file}.ipynb)"
    if pr:
        print(notebook_template)
        pyperclip.copy(notebook_template)
        return

    return notebook_template

In [30]:
gen_notebooke_link()

here's a notebook about `WGAN`, including its formula and visualization: [here](../../notebooks/wgan-cifar10.ipynb)


#### generate the main interested field note

In [8]:
def gen_main_interested_field(pr=True):
    repo_link = input("repo link: ")
    repo_template = f"*Since this is one of the author's `main interested fields`, you may refer to another individual repo for this tech: [here]({repo_link})*"
    if pr:
        print(repo_template)
        pyperclip.copy(repo_template)
        return

    return repo_template

In [ ]:
gen_main_interested_field()

#### generate a paper resource pipeline

In [9]:
def gen_paper_resource_pipeline(pr=True):
    
    url = input('google scholar url: ')
    resouce_title, paper_link, citation = get_from_google_scholar(url)

    resource_title = gen_resource_title(pr=False, resource_title=resouce_title)
    paper_link = gen_paper_link(pr=False, paper_link=paper_link, citation=citation)

    pipe = resource_title + "\n" + paper_link
    if pr:
        print(pr)
        pyperclip.copy(pipe)
        return

    return pipe

In [11]:
gen_paper_resource_pipeline()

True
